#  Numba vectorize

<div class="dateauthor">
21 June 2022 | Jan H. Meinke
</div>

<a href="https://colab.research.google.com/github/Ziaeemehr/workshop_hpcpy/blob/main/notebooks/numba/numba_vectorize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Numba offers a decorator `@vectorize` that allows us to generate **fast** [ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html). 

In [1]:
import numba
import numpy
import numpy as np
import matplotlib.pyplot as plt

## A simple trig function

Let's implement a simple trig function:

In [2]:
import math

In [3]:
def sinacosb(a, b):
    """Calculate the product of sin(a) and cos(b)"""
    return math.sin(a) * math.cos(b)

## Passing numpy arrays as arguments

In [4]:
n = 1000000
a = np.ones(n, dtype='int8')
b = 2 * a

In [5]:
# sinacosb(a,b) # error

The function sinasinb is only defined for scalars, so we have to do something if we want to pass an array.

## numpy.vectorize

NumPy provides the function `vectorize`.

In [6]:
sinacosb_numpy = np.vectorize(sinacosb)

In [7]:
%timeit sinacosb_numpy(a,b)

265 ms ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## numba.vectorize

### Dynamic ufuncs

In [9]:
sinacosb_numba_dynamic = numba.vectorize(sinacosb)

In [10]:
%timeit sinacosb_numba_dynamic(a,b)

21.2 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The function usinacosb is a *dynamic ufunc*. The arguments are determined when the function is called and only then is the function compiled.

### Eager compilation

Assume, we know with what kind of arguments a function is called, then numba can generate code as soon as we call numba vectorize. The decorator can take a list of [type specification](https://numba.readthedocs.io/en/stable/reference/types.html#signatures) strings of the form `f8(f8, f8)`, where the type before the parentheses is the return type and the types within the parentheses are the argument types.

Here, `f8` represents 64-bit float (double precision), `f4` represents 32-bit float (single precision), `i8` represents 64-bit integer, etc.

In [11]:
@numba.vectorize(['f8(i8,i8)', 'f4(f4,f4)', 'f8(f8,f8)'], nopython=True)
def sinacosb_numba_eager(a,b):
    return math.sin(a) * math.cos(b)

### target

If I use eager compilation I can give an addition keyword argument: *target*.
- target="cpu": default, run in a single thread on the CPU
- target="parallel": run in multiple threads
- target="cuda": run on a CUDA-capable GPU

In [12]:
sinacosb_numba_parallel = numba.vectorize(['f8(i8,i8)', 'f4(f4,f4)', 'f8(f8,f8)', ], nopython=True, target="parallel")(sinacosb)

In [13]:
%timeit sinacosb_numba_parallel(a,b)

The slowest run took 717.92 times longer than the fastest. This could mean that an intermediate result is being cached.
767 ms ± 1.18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


The parallel version is slower here because:

- **Overhead of parallelism**: For relatively small arrays (1 million elements) and simple operations, the cost of setting up multiple threads, synchronizing them, and managing the parallel execution can exceed the benefits of parallel computation.

- **Compilation and caching**: The first run includes compilation time, and the variability in timings suggests that intermediate results are being cached, but the parallel target adds extra overhead during compilation and execution.

- **Data size**: With only 1 million elements, the computation is not computationally intensive enough to benefit from parallelism. Parallelism shines for larger datasets or more complex operations.

For larger arrays (e.g., 100 million elements), parallelism can provide significant speedups, as seen in the next cell.

In [14]:
n = 100_000_000
a = np.ones(n, dtype='int8')
b = 2 * a

In [15]:
%timeit sinacosb_numba_eager(a, b)
%timeit sinacosb_numba_parallel(a, b)

4.42 s ± 95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.16 s ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.16 s ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exercise: The Mandelbrot set

The Mandelbrot set is the set of points *c* in the complex plane for which

$$z_{i+1} = z_i^2 + c$$

does not diverge.

The series diverges if $|z_i|>2$ for any *i*.

Since it is impracticable to calculate an infinite number of iterations, one usually sets an upper limit for the number of iterations, for example, 20.

### Escape time algorithm

A simple implementation of this algorithm is the following:

In [16]:
def escape_time(p, maxtime):
    """Perform the Mandelbrot iteration until it's clear that p diverges
    or the maximum number of iterations has been reached.
    
    Parameters
    ----------
    p: complex
        point in the complex plane
    maxtime: int
        maximum number of iterations to perform before p is considered in 
        the Mandelbrot set.
    """
    z = 0j
    for i in range(maxtime):
        z = z ** 2 + p
        if abs(z) > 2:
            return i
    return maxtime

### Todo:

1. Generate a grid of size n times m of complex numbers with the real part taken from the interval [-2.2, 1.5] and the imaginary part taken from the interval [-1.5, 1.5]. Hint, numpy.meshgrid can help.

2. Vectorize escape_time using numba.vectorize and apply it to the array above. Note, the  output is an integer.

3. Visualize the generated array using matplotlib.pyplot.imshow. 


In [ ]:
%timeit M = escape_time_vec(P, 50)